# 7. Function Decorators and Closure

## 7.1 데커레이터

**데커레이터(Decorator)**란?  
소스 코드에 있는 함수를 '표시'해서 함수의 작동을 개선할 수 있게 해 주는 것.

In [1]:
# Temporary
def decorate(func) :
    def ret() : pass
    return ret

In [2]:
@decorate
def target() :
    print('running target()')

위의 코드는 아래 코드와 동일한 동작을 한다.

In [3]:
def target() :
    print('running target()')

target = decorate(target)

데커레이터는 문법적 설탕(Syntactic Sugar)일 뿐이지만, 런타임에 프로그램 행위를 변경하는 **메타프로그래밍**을 할 때 편리하게 쓰인다.  

데커레이터의 핵심 특징은 데커레이터된 함수가 정의된 직후, 즉 **임포트 타임**에 실행된다는 것이다.

데커레이터를 이용하여 전략 패턴을 더욱 개선할 수 있다. (예시는 책에 나와 있으니 따로 언급하지 않는다.)

## 7.2 변수 범위 규칙

In [4]:
def f1(a) :
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [6]:
b = 1
def f2(a) :
    print(a)
    print(b)

f2(3)

3
1


In [8]:
b = 1
def f3(a) :
    print(a)
    print(b)
    b = 3

f3(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

위 예제에서 보듯, 파이썬은 변수가 선언되어 있기를 요구하지는 않지만 함수 본체 안에서 할당된 변수는 지역 변수로 판단한다.  
마지막 예시를 우리가 의도한 대로 동작하게 하려면, **global**이라는 키워드를 사용해서 b를 전역 변수로 다루어야 한다.

In [9]:
b = 1
def f3(a) :
    global b
    print(a)
    print(b)
    b = 3

f3(3)

3
1


## 7.3 클로저
### 가. 클로저

**클로저(Closure)**란?  
함수 본체에서 정의하지 않고 참조하는 비전역 변수를 포함한 확장 범위를 가진 함수를 의미한다.  
익명 함수와 혼동하는 사람이 많으나, 익명 함수인지 아닌지는 중요한 것이 아니다.

In [11]:
def make_averager() :
    series = []
    
    def averager(new_val) :
        series.append(new_val)
        total = sum(series)
        return total / len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


위 코드에서 averager 내의 series 변수는 **자유 변수(Free Variable)**이다. 다른 말로, 지역 범위에 바인딩되어 있지 않은 변수다.

### 나. Nonlocal

사실 위의 코드는 매번 수의 개수와 총합을 다시 계산하므로 비효율적인 코드이다. 아래와 같이 바꾸면 어떨까?

In [12]:
def make_averager() :
    count = 0
    total = 0
    
    def averager(new_val) :
        count += 1
        total += new_val
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))

UnboundLocalError: local variable 'count' referenced before assignment

averager 내에서 count에 할당이 일어났기 때문에 count를 지역 변수로 인식했고, 그래서 오류가 발생하였다.  
이를 고치기 위해서는, **nonlocal** 키워드를 이용하면 된다. 이는 해당 변수가 자유 변수임을 나타내는 선언이다.

In [14]:
def make_averager() :
    count = 0
    total = 0
    
    def averager(new_val) :
        nonlocal count, total
        count += 1
        total += new_val
        return total / count
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


## 7.4 데커레이터 누적 및 매개변수화
### 가. 누적된 데커레이터

In [16]:
# Temporary
def d1(func) :
    def ret() : pass
    return ret

def d2(func) :
    def ret() : pass
    return ret

In [17]:
@d1
@d2
def f() : pass

위의 코드는 아래와 같다. (d1과 d2의 순서에 주의하라.)

In [19]:
def f() : pass
f = d1(d2(f))

### 나. 매개변수화된 데커레이터

데커레이터에 매개변수를 넣을 수 있다!  

인수를 받아 데커레이터를 반환하는 **데커레이터 팩토리**를 만들고 나서, 데커레이트될 함수에 데커레이터 팩토리를 적용하는 방식으로 하면 된다.

# 8. Object References, Mutability and Recycling

## 8.1 변수, 정체성, 동질성, 별명
### 가. 변수는 상자가 아니다

파이썬에서 변수를 상자로 이해하는 것보다는 객체에 붙은 '레이블'로서 이해하는 것이 좋다.  
* 객체는 변수보다 먼저 생성된다.

In [20]:
class Temp :
    def __init__(self) : pass

x = Temp()
y = Temp() * 10

TypeError: unsupported operand type(s) for *: 'Temp' and 'int'

### 나. 정체성, 동질성, 별명

모든 객체는 정체성, 자료형, 값을 가지고 있다. 객체의 정체성은 일단 생성한 후에는 결코 변경되지 않는다. 정체성은 메모리 내의 객체 주소라고 생각할 수 있다. is 연산자는 두 객체의 '정체성'을 비교한다. id() 함수는 정체성을 나타내는 정수를 반환한다.  
* 참고로 id 값의 실제 의미는 구현에 따라 다르다.


아래 코드는 **정체성**과 **동질성**의 차이를 명확하게 보여 준다.

In [24]:
a = [1, 2, 3]
b = a
print(a is b)

True


In [25]:
c = [1, 2, 3]
print(a == c)  # 동질성
print(a is c)  # 정체성

True
False


## 8.2 얕은 복사와 깊은 복사

In [31]:
l1 = [1, [10, 20, 30], (10, 20, 30)]
l2 = list(l1)
print(l1 == l2)
print(l1 is l2)

True
False


기본적으로 파이썬에서 생성자를 사용하면 얕은 복사가 이루어진다. 그러나 가변 항목이 들어 있을 경우 조심스럽게 사용해야 한다. (아래 코드 참조)

In [32]:
l1.append(100)
l1[1].remove(20)
print('l1:', l1)
print('l2:', l2)

l1: [1, [10, 30], (10, 20, 30), 100]
l2: [1, [10, 30], (10, 20, 30)]


In [33]:
l2[1] += [50, 70]
l2[2] += (40, 50)
print('l1:', l1)
print('l2:', l2)

l1: [1, [10, 30, 50, 70], (10, 20, 30), 100]
l2: [1, [10, 30, 50, 70], (10, 20, 30, 40, 50)]


copy 모듈의 copy()는 얕은 복사를, deepcopy()는 깊은 복사를 지원한다.

## 8.3 참조로서의 함수 매개변수

파이썬은 **공유로 호출**하는 매개변수 전달 방식만 지원한다. 이는 함수의 각 매개변수가 인수로 전달받은 각 참조의 사본을 받는다는 의미다. 달리 말하면, 함수 안의 매개변수는 실제 인수의 별명이 된다.

이러한 특성으로 인해, 가변형을 매개변수 기본값으로 사용하는 것은 의도치 않은 동작을 야기할 수 있으므로 지양해야 한다.